In [1]:
pwd()

"/Users/alexpanchot/Documents/Caller_preds/caller_predictions/prelim/all_log"

In [5]:
import CSV
using DataFrames
# using StatsBase
include("$(dirname(dirname(pwd())))/pred_funcs.jl")
using MLJ
# import MLJBase
import TableView.showtable
using Plots
import FileIO
import Statistics
import LinearAlgebra
using StatsPlots
using Random
Random.seed!(0);
using StatsBase
import MLJBase

WebIO._IJuliaInit()

In [6]:
MissingImputator = @load MissingImputator pkg="BetaML" verbosity=0
LogisticClassifier = @load LogisticClassifier pkg="MLJLinearModels" verbosity=0;  # loads code defining a model type
EvoTreeClassifier = @load EvoTreeClassifier verbosity=0

EvoTrees.EvoTreeClassifier

In [7]:
full = ""

""

In [11]:

sample_key2, xold, yold = FileIO.load("$(dirname(dirname(pwd())))/data$(full).jld2","sample_key","x" ,"y");


xold.col1=collect(1:nrow(xold));
println(nrow(xold))
# filter!(x->x.ALTREF_I ≈ 0,xold)
println(nrow(xold))
# filter!(x->x.ALT_I>0.5 || x.REF_I>0.5,xold) # indel
# filter!(x->x.ALT_I<0.5 && x.REF_I<0.5,xold) # SNV
println(nrow(xold))
filter!(x->x.training>0.5,xold)
# select!(xold,Not("training"));
println(nrow(xold))
# filter!(x->!ismissing(x.dust_score_Mutect),xold);
# println(nrow(xold))

yold = yold[xold.col1]
# println(xold[findall(x->x,[i==true for i in yold]),"col1"])
# println(xold[findall(x->!x,[i==true for i in yold]),"col1"])
sample_key2 = sample_key2[xold.col1]
select!(xold,Not("col1"));

94626
94626
94626
26217


In [12]:
for j in 1:ncol(xold), i in 1:nrow(xold)
    if !ismissing(xold[i,j]) && (isnan(xold[i,j]) || isinf(xold[i,j]) )
        xold[i,j] = missing
    end
end

In [13]:
train2, test2 = partition(eachindex(yold), 0.5, stratify=yold,  shuffle=true, rng=12344);

select!(xold,Not("training"));

In [14]:
sum([i==true for i in yold[train2]])

46

In [15]:
sum([i==true for i in yold[test2]])

46

In [16]:
sum(.~ismissing.(xold[train2,:dust_score_Vardict]))

8023

In [17]:
FileIO.save("testdata$(full).jld2", Dict("sample_key"=>sample_key2[test2],"x" => xold[test2,:],
        "y" => yold[test2], 
    ) ; compress = true
)
FileIO.save("traindata$(full).jld2", Dict("sample_key"=>sample_key2[train2],"x" => xold[train2,:],
        "y" => yold[train2], 
    ) ; compress = true
)

# Mutect

In [33]:
x = copy(xold[train2,:]);
y = copy(yold[train2]);

In [34]:
x.col1=collect(1:nrow(x));
coldrop = [] #"FPpass","FILTER_Mutect_PASS"] # ["gt_AF_Mutect","AltFwd_Mutect","RefFwd_Mutect","AltRev_Mutect","RefRev_Mutect","gt_AD_ref_Mutect","gt_AD_alt_Mutect"]
for i in names(x)
    if i == "col1" || i=="training" || occursin( "FILTER_Mutect",i)
        continue
    end
    if length(unique(x[:,i])) < 2 || occursin("Pindel",i) || occursin("Lofreq",i) || occursin("Vardict",i) # || occursin("ALT",i) || occursin("REF",i)
        push!(coldrop,i)
#         println(i)
    end 
end
unique!(coldrop)
select!(x, Not(coldrop));
println(sum([i==true for i in y]))
# println(nrow(xold))
# dropmissing!(xold)
# println(nrow(xold))
filter!(x->!ismissing(x.dust_score_Mutect),x);
println(size(y))
# println(length(sample_key2))
println()
y = y[x.col1]
# println(xold[findall(x->x,[i==true for i in yold]),"col1"])
# println(xold[findall(x->!x,[i==true for i in yold]),"col1"])
# sample_key2 = sample_key2[x.col1]
select!(x,Not("col1"));

println(size(x))
println(size(y))
# println(length(sample_key2))
println(sum([i==true for i in y]));


46
(13108,)

(817, 70)
(817,)
39


In [35]:
transform_log!(x,10)

In [36]:
stnd = fit_standardizer(x)
transform_standardizer!(x,stnd);

In [37]:
println(sum(ismissing.(Matrix(x))))

18


In [38]:
imputemach = machine(MissingImputator(K=5),x) 
@time fit!(imputemach,verbosity=1)

┌ Info: Training Machine{MissingImputator,…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403


  1.048704 seconds (1.78 M allocations: 317.766 MiB, 47.18% gc time)


Machine{MissingImputator,…} trained 1 time; caches data
  args: 
    1:	Source @810 ⏎ `Table{Union{AbstractVector{Union{Missing, Continuous}}, AbstractVector{Continuous}}}`


In [39]:
tt = MLJ.transform(imputemach,x);
for i in 1:ncol(x)
    x[:,i] = tt[i]
end

In [40]:
disallowmissing!(x);

In [41]:
println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))


(817, 70) 39
(817,)


In [42]:
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"under",0.8);
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"over",0.2);

In [43]:
y = coerce(copy(y),OrderedFactor);

In [44]:
println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))

(817, 70) 39
(817,)


In [45]:
r = [range(LogisticClassifier(), :fit_intercept, values=[true,false]),
    range(LogisticClassifier(), :penalize_intercept, values=[true,false]),
    range(LogisticClassifier(), :penalty, values=[:none,:l1,:l2]),
    range(LogisticClassifier(), :lambda, lower=0.0,upper=1.0),
    range(LogisticClassifier(), :gamma, lower=0.0,upper=1.0),
]

5-element Vector{MLJBase.ParamRange}:
 NominalRange(fit_intercept = true, false)
 NominalRange(penalize_intercept = true, false)
 NominalRange(penalty = none, l1, l2)
 NumericRange(0.0 ≤ lambda ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ gamma ≤ 1.0; origin=0.5, unit=0.5)

In [46]:
self_tune = TunedModel(
#     model=LogisticClassifier(penalty=:l1),
    model=LogisticClassifier(),
    
    resampling=StratifiedCV(nfolds=5, rng=123),

    tuning=LatinHypercube(),  #     tuning=Grid(resolution=2), 
    range = r,
    n=200,
    operation=predict_mode, # use for probabalistic 
    measure = fnr,  # use for deterministic or with predict_mode for probab 
    acceleration=CPUThreads(),
)

machbest = machine(
    self_tune,
    x,y)
fit!(machbest,verbosity=1 )



┌ Info: Training Machine{ProbabilisticTunedModel{LatinHypercube,…},…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403
┌ Info: Attempting to evaluate 200 models.
└ @ MLJTuning /Users/alexpanchot/.julia/packages/MLJTuning/efiDR/src/tuned_models.jl:685
Evaluating over 200 metamodels:   0%[>                        ]  ETA: N/A┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Eva

Evaluating over 200 metamodels:  22%[=====>                   ]  ETA: 0:00:42┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinea

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge 

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge 

Evaluating over 200 metamodels:  62%[===============>         ]  ETA: 0:00:17┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  62%[===============>         ]  ETA: 0:00:17┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 100

Evaluating over 200 metamodels:  74%[==================>      ]  ETA: 0:00:11┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  74%[==================>      ]  ETA: 0:00:11┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 100

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  83%[====================>    ]  ETA: 0:00:07┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  84%[====================>    ]  ETA: 0:00:07┌ Warning: Proximal GD did not converge in 100

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  99%[========================>]  ETA: 0:00:00┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinea

Machine{ProbabilisticTunedModel{LatinHypercube,…},…} trained 1 time; caches data
  args: 
    1:	Source @024 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @934 ⏎ `AbstractVector{OrderedFactor{2}}`


In [47]:
mach = machine(
    fitted_params(machbest).best_model,
    x,y)
fit!(mach,verbosity=4 )

┌ Info: Training Machine{LogisticClassifier,…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403


Machine{LogisticClassifier,…} trained 1 time; caches data
  args: 
    1:	Source @030 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @535 ⏎ `AbstractVector{OrderedFactor{2}}`


In [48]:
FileIO.save("mutect$(full).jld2", Dict("mach"=>mach,"stnd" => stnd,
        "imputemach" => imputemach, 
    ) ; compress = true
)

# Vardict

In [49]:
x = copy(xold[train2,:]);
y = copy(yold[train2]);

In [50]:
x.col1=collect(1:nrow(x));
coldrop = [] #"FPpass","FILTER_Mutect_PASS"] # ["gt_AF_Mutect","AltFwd_Mutect","RefFwd_Mutect","AltRev_Mutect","RefRev_Mutect","gt_AD_ref_Mutect","gt_AD_alt_Mutect"]
for i in names(x)
    if i == "col1" || i=="training" || occursin( "FILTER_Vardict",i)
        continue
    end
    if length(unique(x[:,i])) < 2 || occursin("Pindel",i) || occursin("Lofreq",i) || occursin("Mutect",i) # || occursin("ALT",i) || occursin("REF",i)
        push!(coldrop,i)
#         println(i)
    end 
end
unique!(coldrop)
select!(x, Not(coldrop));
println(sum([i==true for i in y]))
# println(nrow(xold))
# dropmissing!(xold)
# println(nrow(xold))
filter!(x->!ismissing(x.dust_score_Vardict),x);
println(size(y))
# println(length(sample_key2))
println()
y = y[x.col1]
# println(xold[findall(x->x,[i==true for i in yold]),"col1"])
# println(xold[findall(x->!x,[i==true for i in yold]),"col1"])
# sample_key2 = sample_key2[x.col1]
select!(x,Not("col1"));

println(size(x))
println(size(y))
# println(length(sample_key2))
println(sum([i==true for i in y]));



46
(13108,)

(8023, 63)
(8023,)
46


In [51]:
transform_log!(x,10)
stnd = fit_standardizer(x)
transform_standardizer!(x,stnd);
println(sum(ismissing.(Matrix(x))))

1788


In [52]:
imputemach = machine(MissingImputator(K=5),x) 
@time fit!(imputemach,verbosity=1)

┌ Info: Training Machine{MissingImputator,…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403


  6.017921 seconds (34.97 M allocations: 5.406 GiB, 19.04% gc time, 2.34% compilation time)


Machine{MissingImputator,…} trained 1 time; caches data
  args: 
    1:	Source @790 ⏎ `Table{Union{AbstractVector{Union{Missing, Continuous}}, AbstractVector{Continuous}}}`


In [53]:
tt = MLJ.transform(imputemach,x);
for i in 1:ncol(x)
    x[:,i] = tt[i]
end
disallowmissing!(x);

In [54]:
println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"under",0.8);
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"over",0.2);

y = coerce(copy(y),OrderedFactor);

println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))

(8023, 63) 46
(8023,)
(8023, 63) 46
(8023,)


In [55]:
r = [range(LogisticClassifier(), :fit_intercept, values=[true,false]),
    range(LogisticClassifier(), :penalize_intercept, values=[true,false]),
    range(LogisticClassifier(), :penalty, values=[:none,:l1,:l2]),
    range(LogisticClassifier(), :lambda, lower=0.0,upper=1.0),
    range(LogisticClassifier(), :gamma, lower=0.0,upper=1.0),
]

5-element Vector{MLJBase.ParamRange}:
 NominalRange(fit_intercept = true, false)
 NominalRange(penalize_intercept = true, false)
 NominalRange(penalty = none, l1, l2)
 NumericRange(0.0 ≤ lambda ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ gamma ≤ 1.0; origin=0.5, unit=0.5)

In [56]:
self_tune = TunedModel(
#     model=LogisticClassifier(penalty=:l1),
    model=LogisticClassifier(),
    
    resampling=StratifiedCV(nfolds=5, rng=123),

    tuning=LatinHypercube(),  #     tuning=Grid(resolution=2), 
    range = r,
    n=200,
    operation=predict_mode, # use for probabalistic 
    measure = fnr,  # use for deterministic or with predict_mode for probab 
    acceleration=CPUThreads(),
)

machbest = machine(
    self_tune,
    x,y)
fit!(machbest,verbosity=1 )

mach = machine(
    fitted_params(machbest).best_model,
    x,y)
fit!(mach,verbosity=1 )


┌ Info: Training Machine{ProbabilisticTunedModel{LatinHypercube,…},…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403
┌ Info: Attempting to evaluate 200 models.
└ @ MLJTuning /Users/alexpanchot/.julia/packages/MLJTuning/efiDR/src/tuned_models.jl:685
Evaluating over 200 metamodels:   0%[>                        ]  ETA: N/A┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Eva

Evaluating over 200 metamodels:  15%[===>                     ]  ETA: 0:04:12┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinea

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge 

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge 

Evaluating over 200 metamodels:  56%[==============>          ]  ETA: 0:02:13┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  56%[==============>          ]  ETA: 0:02:11┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  57%[=========

Evaluating over 200 metamodels:  72%[==================>      ]  ETA: 0:01:17┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  72%[==================>      ]  ETA: 0:01:16┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 100

Evaluating over 200 metamodels:  86%[=====================>   ]  ETA: 0:00:38┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  86%[=====================>   ]  ETA: 0:00:37┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  86%[=====================>   ]  ETA: 0:00:35┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModel

Machine{LogisticClassifier,…} trained 1 time; caches data
  args: 
    1:	Source @215 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @697 ⏎ `AbstractVector{OrderedFactor{2}}`


In [57]:
FileIO.save("vardict$(full).jld2", Dict("mach"=>mach,"stnd" => stnd,
        "imputemach" => imputemach, 
    ) ; compress = true
)

# Lofreq

In [58]:
x = copy(xold[train2,:]);
y = copy(yold[train2]);

In [59]:
x.col1=collect(1:nrow(x));
coldrop = [] #"FPpass","FILTER_Mutect_PASS"] # ["gt_AF_Mutect","AltFwd_Mutect","RefFwd_Mutect","AltRev_Mutect","RefRev_Mutect","gt_AD_ref_Mutect","gt_AD_alt_Mutect"]
for i in names(x)
    if i == "col1" || i=="training" || occursin( "FILTER_Lofreq",i)
        continue
    end
    if length(unique(x[:,i])) < 2 || occursin("Pindel",i) || occursin("Vardict",i) || occursin("Mutect",i) # || occursin("ALT",i) || occursin("REF",i)
        push!(coldrop,i)
#         println(i)
    end 
end
unique!(coldrop)
select!(x, Not(coldrop));
println(sum([i==true for i in y]))
# println(nrow(xold))
# dropmissing!(xold)
# println(nrow(xold))
filter!(x->!ismissing(x.dust_score_Lofreq),x);
println(size(y))
# println(length(sample_key2))
println()
y = y[x.col1]
# println(xold[findall(x->x,[i==true for i in yold]),"col1"])
# println(xold[findall(x->!x,[i==true for i in yold]),"col1"])
# sample_key2 = sample_key2[x.col1]
select!(x,Not("col1"));

println(size(x))
println(size(y))
# println(length(sample_key2))
println(sum([i==true for i in y]));




46
(13108,)

(10016, 50)
(10016,)
46


In [60]:
transform_log!(x,10)
stnd = fit_standardizer(x)
transform_standardizer!(x,stnd);
println(sum(ismissing.(Matrix(x))))

1201


In [61]:
imputemach = machine(MissingImputator(K=5),x) 
@time fit!(imputemach,verbosity=1)

┌ Info: Training Machine{MissingImputator,…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403


 12.307647 seconds (20.52 M allocations: 2.218 GiB, 3.73% gc time, 85.30% compilation time)


Machine{MissingImputator,…} trained 1 time; caches data
  args: 
    1:	Source @117 ⏎ `Table{Union{AbstractVector{Union{Missing, Continuous}}, AbstractVector{Continuous}}}`


In [62]:
tt = MLJ.transform(imputemach,x);
for i in 1:ncol(x)
    x[:,i] = tt[i]
end
disallowmissing!(x);

In [63]:
println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"under",0.8);
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"over",0.2);

y = coerce(copy(y),OrderedFactor);

println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))

(10016, 50) 46
(10016,)
(10016, 50) 46
(10016,)


In [64]:
r = [range(LogisticClassifier(), :fit_intercept, values=[true,false]),
    range(LogisticClassifier(), :penalize_intercept, values=[true,false]),
    range(LogisticClassifier(), :penalty, values=[:none,:l1,:l2]),
    range(LogisticClassifier(), :lambda, lower=0.0,upper=1.0),
    range(LogisticClassifier(), :gamma, lower=0.0,upper=1.0),
]

5-element Vector{MLJBase.ParamRange}:
 NominalRange(fit_intercept = true, false)
 NominalRange(penalize_intercept = true, false)
 NominalRange(penalty = none, l1, l2)
 NumericRange(0.0 ≤ lambda ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ gamma ≤ 1.0; origin=0.5, unit=0.5)

In [65]:
self_tune = TunedModel(
#     model=LogisticClassifier(penalty=:l1),
    model=LogisticClassifier(),
    
    resampling=StratifiedCV(nfolds=5, rng=123),

    tuning=LatinHypercube(),  #     tuning=Grid(resolution=2), 
    range = r,
    n=200,
    operation=predict_mode, # use for probabalistic 
    measure = fnr,  # use for deterministic or with predict_mode for probab 
    acceleration=CPUThreads(),
)

machbest = machine(
    self_tune,
    x,y)
fit!(machbest,verbosity=1 )

mach = machine(
#     LogisticClassifier(penalty=:l1),
    fitted_params(machbest).best_model,
    x,y)
fit!(mach,verbosity=1 )

┌ Info: Training Machine{ProbabilisticTunedModel{LatinHypercube,…},…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403
┌ Info: Attempting to evaluate 200 models.
└ @ MLJTuning /Users/alexpanchot/.julia/packages/MLJTuning/efiDR/src/tuned_models.jl:685
Evaluating over 200 metamodels:   0%[>                        ]  ETA: N/A┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:   0%[>                        ]  ETA: 0:39:01┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge 

Evaluating over 200 metamodels:  30%[=======>                 ]  ETA: 0:03:42┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinea

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge 

Evaluating over 200 metamodels:  70%[=================>       ]  ETA: 0:01:38┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  71%[=================>       ]  ETA: 0:01:31┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  72%[=================>       ]  ETA: 0:01:30┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  72%[==================>      ]  ETA: 0:01:28┌ Warning: Proximal GD did not converge in 1000 iter

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  80%[====================>    ]  ETA: 0:01:05┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinea

Evaluating over 200 metamodels:  88%[=====================>   ]  ETA: 0:00:42┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  88%[======================>  ]  ETA: 0:00:40┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  88%[======================>  ]  ETA: 0:00:38┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  90%[======================>  ]  ETA: 0:00:34┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iter

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  99%[========================>]  ETA: 0:00:03┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels: 100%[========================>]  ETA: 0:00:02┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels: 100%[=========

Machine{LogisticClassifier,…} trained 1 time; caches data
  args: 
    1:	Source @198 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @983 ⏎ `AbstractVector{OrderedFactor{2}}`


In [66]:
FileIO.save("lofreq$(full).jld2", Dict("mach"=>mach,"stnd" => stnd,
        "imputemach" => imputemach, 
    ) ; compress = true
)

# Pindel

In [67]:
x = copy(xold[train2,:]);
y = copy(yold[train2]);

In [68]:
x.col1=collect(1:nrow(x));
coldrop = [] #"FPpass","FILTER_Mutect_PASS"] # ["gt_AF_Mutect","AltFwd_Mutect","RefFwd_Mutect","AltRev_Mutect","RefRev_Mutect","gt_AD_ref_Mutect","gt_AD_alt_Mutect"]
for i in names(x)
    if i == "col1" || i=="training" || occursin( "FILTER_Pindel",i)
        continue
    end
    if length(unique(x[:,i])) < 2 || occursin("Lofreq",i) || occursin("Vardict",i) || occursin("Mutect",i) # || occursin("ALT",i) || occursin("REF",i)
        push!(coldrop,i)
#         println(i)
    end 
end
unique!(coldrop)
select!(x, Not(coldrop));
println(sum([i==true for i in y]))
# println(nrow(xold))
# dropmissing!(xold)
# println(nrow(xold))
filter!(x->!ismissing(x.dust_score_Pindel),x);
println(size(y))
# println(length(sample_key2))
println()
y = y[x.col1]
# println(xold[findall(x->x,[i==true for i in yold]),"col1"])
# println(xold[findall(x->!x,[i==true for i in yold]),"col1"])
# sample_key2 = sample_key2[x.col1]
select!(x,Not("col1"));

println(size(x))
println(size(y))
# println(length(sample_key2))
println(sum([i==true for i in y]));

46
(13108,)

(2530, 40)
(2530,)
5


In [69]:
transform_log!(x,10)
stnd = fit_standardizer(x)
transform_standardizer!(x,stnd);
println(sum(ismissing.(Matrix(x))))

1859


In [70]:
imputemach = machine(MissingImputator(K=5),x) 
@time fit!(imputemach,verbosity=1)

┌ Info: Training Machine{MissingImputator,…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403


  1.771257 seconds (13.70 M allocations: 1.515 GiB, 16.00% gc time, 10.08% compilation time)


Machine{MissingImputator,…} trained 1 time; caches data
  args: 
    1:	Source @762 ⏎ `Table{Union{AbstractVector{Union{Missing, Continuous}}, AbstractVector{Continuous}}}`


In [71]:
tt = MLJ.transform(imputemach,x);
for i in 1:ncol(x)
    x[:,i] = tt[i]
end
disallowmissing!(x);

In [72]:
println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"under",0.8);
# x,y,train,test,sample_key = sampler(x,y,train,test,sample_key,"over",0.2);

y = coerce(copy(y),OrderedFactor);

println(size(x)," ",sum([i==true for i in y] ) )
println(size(y))

(2530, 40) 5
(2530,)
(2530, 40) 5
(2530,)


In [73]:
r = [range(LogisticClassifier(), :fit_intercept, values=[true,false]),
    range(LogisticClassifier(), :penalize_intercept, values=[true,false]),
    range(LogisticClassifier(), :penalty, values=[:none,:l1,:l2]),
    range(LogisticClassifier(), :lambda, lower=0.0,upper=1.0),
    range(LogisticClassifier(), :gamma, lower=0.0,upper=1.0),
]

5-element Vector{MLJBase.ParamRange}:
 NominalRange(fit_intercept = true, false)
 NominalRange(penalize_intercept = true, false)
 NominalRange(penalty = none, l1, l2)
 NumericRange(0.0 ≤ lambda ≤ 1.0; origin=0.5, unit=0.5)
 NumericRange(0.0 ≤ gamma ≤ 1.0; origin=0.5, unit=0.5)

In [74]:
self_tune = TunedModel(
#     model=LogisticClassifier(penalty=:l1),
    model=LogisticClassifier(),
    
    resampling=StratifiedCV(nfolds=5, rng=123),

    tuning=LatinHypercube(),  #     tuning=Grid(resolution=2), 
    range = r,
    n=200,
    operation=predict_mode, # use for probabalistic 
    measure = fnr,  # use for deterministic or with predict_mode for probab 
    acceleration=CPUThreads(),
)

machbest = machine(
    self_tune,
    x,y)
fit!(machbest,verbosity=1 )

mach = machine(
    fitted_params(machbest).best_model,
    x,y)
fit!(mach,verbosity=1 )

┌ Info: Training Machine{ProbabilisticTunedModel{LatinHypercube,…},…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403
┌ Info: Attempting to evaluate 200 models.
└ @ MLJTuning /Users/alexpanchot/.julia/packages/MLJTuning/efiDR/src/tuned_models.jl:685
Evaluating over 200 metamodels:   0%[>                        ]  ETA: N/A┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ W

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge 

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge 

Evaluating over 200 metamodels:  53%[=============>           ]  ETA: 0:00:22┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinea

Evaluating over 200 metamodels:  74%[==================>      ]  ETA: 0:00:11┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  74%[==================>      ]  ETA: 0:00:11┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  75%[=========

Evaluating over 200 metamodels:  88%[======================>  ]  ETA: 0:00:05┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels:  88%[======================>  ]  ETA: 0:00:05┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
┌ Warning: Proximal GD did not converge in 100

┌ Warning: Proximal GD did not converge in 1000 iterations.
└ @ MLJLinearModels /Users/alexpanchot/.julia/packages/MLJLinearModels/ISbgA/src/fit/proxgrad.jl:64
Evaluating over 200 metamodels: 100%[=========================] Time: 0:00:47
┌ Info: Training Machine{LogisticClassifier,…}.
└ @ MLJBase /Users/alexpanchot/.julia/packages/MLJBase/HZmTU/src/machines.jl:403


Machine{LogisticClassifier,…} trained 1 time; caches data
  args: 
    1:	Source @358 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @311 ⏎ `AbstractVector{OrderedFactor{2}}`


In [75]:
FileIO.save("pindel$(full).jld2", Dict("mach"=>mach,"stnd" => stnd,
        "imputemach" => imputemach, 
    ) ; compress = true
)